In [1]:
import glob
import gzip
import os

import pandas as pd
import scanpy as sc
import numpy as np
import scipy.io
import scipy.sparse

In [2]:
matrix = "../../data/01ssc-bal/fastcar/SSc_SSc12/"
sct_counts = "../../data/01ssc-bal/sct/SSc_SSc12/log-counts.mtx"
sct_hvg = "../../data/01ssc-bal/sct/SSc_SSc12/hvg-scaled.tsv"

In [3]:
sample="SSc_SSc12"
min_genes=200
min_cells=5
n_pcs=40
resolution=0.5

In [4]:
ds = sc.read_10x_mtx(matrix, var_names="gene_symbols", make_unique=False)

In [5]:
ds.var_names_make_unique(join=".")

In [6]:
scaled = pd.read_csv(sct_hvg, sep="\t")

In [7]:
ds = ds[scaled.columns, :].copy()

In [8]:
scaled.columns = ds.obs_names

In [9]:
sc.pp.filter_cells(ds, min_genes=min_genes)

In [10]:
sc.pp.filter_genes(ds, min_cells=min_cells)

In [11]:
counts = scipy.io.mmread(sct_counts)

In [12]:
ds.X = counts.tocsc().T

In [13]:
ds.raw = ds

In [14]:
ds2 = ds[:, ds.var_names.isin(scaled.index)]

In [15]:
ds2.shape

(10167, 2999)

In [20]:
scaled.index[~scaled.index.isin(ds.var_names)]

Index(['RP11-442N24--B.1'], dtype='object')

In [22]:
'RP11-442N24__B.1' in ds.var_names

True

So this is the result of Seurat's implicit and mandatory conversion of all underscores to dashes in gene names. We'll have to mimick it in scanpy 